In [16]:
!python -m pip list | findstr -i "torch tensor transformers"

safetensors                  0.4.5
sentence-transformers        5.0.0
tensorboard                  2.19.0
tensorboard-data-server      0.7.2
tensorflow-io-gcs-filesystem 0.31.0
torch                        2.7.1
transformers                 4.51.3


In [16]:
from sentence_transformers import SentenceTransformer, util

model = SentenceTransformer('paraphrase-multilingual-MiniLM-L12-v2')

In [44]:
sementes = [
    "A agência não me respondeu",
    "Estou sem retorno da agência há dias",
    "Ninguém na agência me atende",
    "Já fui na agência e não obtive ajuda",
    "Já fui na agência e não obtive retorno",
    "Já fui na agência presencialmente e não obtive retorno",
    "Contato com o gerente sem sucesso",
    "Insatisfação com o gerente",
    "Gerente não responde",
    "Gerente não atende"
]

emb_sementes = model.encode(sementes)
emb_sementes

array([[ 0.12102936,  0.08120994,  0.15191577, ...,  0.01538015,
        -0.02975922, -0.20485875],
       [ 0.09367761, -0.11368023,  0.16063368, ..., -0.1482455 ,
        -0.17852655, -0.21378818],
       [-0.02578321, -0.12837312,  0.0571832 , ..., -0.11116888,
        -0.04037835, -0.13879843],
       ...,
       [ 0.15645838,  0.04672366,  0.18105428, ...,  0.02525354,
        -0.31324235,  0.29308414],
       [ 0.01475083,  0.12711722,  0.07797929, ...,  0.10182348,
        -0.0203455 ,  0.13040237],
       [ 0.05750076,  0.11216976,  0.08232014, ..., -0.03714725,
         0.02680157,  0.21167849]], dtype=float32)

In [45]:
def similaridade(texto):
    emb = model.encode([texto])
    score = util.cos_sim(emb, emb_sementes).max().item()
    return score

texto_0 = """
Gostaria de expressar minha frustração em relação a um problema recorrente com meu cartão de crédito. Recentemente, tive meu cartão bloqueado sem aviso prévio, o que me causou diversos transtornos, inclusive a perda de uma compra importante. Fui até a agência para tentar resolver a situação e, apesar do contratempo, fui bem atendido pelos funcionários, que demonstraram cordialidade e disposição para ajudar. No entanto, mesmo com o esforço da equipe local, a resolução foi lenta e burocrática, exigindo diversos contatos com a central de atendimento, que por sua vez, foi ineficiente e contraditória nas informações. Esperava mais agilidade e clareza na comunicação por parte da instituição, especialmente se tratando de um serviço essencial como o cartão de crédito. Agradeço o empenho dos colaboradores da agência, mas reforço a necessidade de melhorias nos processos internos e no suporte ao cliente.
"""

texto_1 = """
Venho por meio desta manifestar minha profunda insatisfação com o atendimento prestado pela agência em questão. Desde o primeiro contato, fui tratado com descaso e falta de profissionalismo. Tive dificuldades para obter informações básicas, os prazos prometidos não foram cumpridos e, em diversas ocasiões, fui transferido entre atendentes sem que meu problema fosse efetivamente resolvido. Além disso, percebi total falta de empatia por parte da equipe, que demonstrou impaciência e despreparo para lidar com as demandas apresentadas. É inaceitável que uma instituição que deveria prezar pela qualidade no atendimento ao cliente mantenha esse tipo de conduta. Estou extremamente frustrado com a experiência e espero que medidas corretivas sejam tomadas com urgência. Caso contrário, tomarei as providências cabíveis junto aos órgãos competentes para garantir meus direitos como consumidor.
"""
print(similaridade(texto_0))
print(similaridade(texto_1))

0.4844748377799988
0.5640543103218079


## Quebrar o texto em sentenças

In [49]:
import nltk, numpy as np

centro = np.mean(model.encode(sementes), axis=0, keepdims=True)

def score_por_frase(texto):
    frases = nltk.sent_tokenize(texto, language='portuguese')
    emb_frases = model.encode(frases)
    sims = util.cos_sim(emb_frases, centro).squeeze()
    idx = int(sims.argmax())          # frase mais parecida
    return float(sims.max()), frases[idx]

score0, melhor_frase0 = score_por_frase(texto_0)
score1, melhor_frase1 = score_por_frase(texto_1)
print(score0, melhor_frase0)
print(score1, melhor_frase1)

0.5177465677261353 Fui até a agência para tentar resolver a situação e, apesar do contratempo, fui bem atendido pelos funcionários, que demonstraram cordialidade e disposição para ajudar.
0.5961281061172485 Tive dificuldades para obter informações básicas, os prazos prometidos não foram cumpridos e, em diversas ocasiões, fui transferido entre atendentes sem que meu problema fosse efetivamente resolvido.
